In [2]:
%load_ext autoreload
%autoreload 2

# Polybench Experimental Results

This notebook puts together how we run Polybench experiments using the `pb-flow` script, and how we process the run results to get the formatted table.

## Setup

We need to put the `python/` directory into the `PYTHONPATH`. 

In [3]:
import sys
import os
import pandas as pd

sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())))

In [4]:
import python.utils.polybench as pb_utils

In [5]:
len(pb_utils.discover_examples('./example/polybench'))

30

In [6]:
options = {
  "baseline": pb_utils.PbFlowOptions(
    pb_dir=os.path.join(pb_utils.get_project_root(), 'example', 'polybench'),
    work_dir=os.path.join(pb_utils.get_project_root(), 'tmp', 'phism', 'pb-flow.medium.baseline'),
    job=30,
    polymer=False,
    cosim=True,
    debug=True,
    dataset='MEDIUM',
    cleanup=False,
    loop_transforms=False,
    improve_pipelining=False,
  ),
  "polymer": pb_utils.PbFlowOptions(
    pb_dir=os.path.join(pb_utils.get_project_root(), 'example', 'polybench'),
    work_dir=os.path.join(pb_utils.get_project_root(), 'tmp', 'phism', 'pb-flow.medium.polymer'),
    job=30,
    polymer=True,
    cosim=True,
    debug=True,
    dataset='MEDIUM',
    cleanup=False,
    loop_transforms=True,
    max_span=1,
  ),
  "polymer-baseline": pb_utils.PbFlowOptions(
    pb_dir=os.path.join(pb_utils.get_project_root(), 'example', 'polybench'),
    work_dir=os.path.join(pb_utils.get_project_root(), 'tmp', 'phism', 'pb-flow.medium.polymer-baseline'),
    job=30,
    polymer=True,
    cosim=True,
    debug=True,
    dataset='MEDIUM',
    cleanup=False,
    loop_transforms=False,
    # constant_args=False,
    improve_pipelining=False,
    max_span=-1,
  ),
}

## Run `pb-flow`

In [9]:
pb_utils.pb_flow_runner(options["baseline"])

>>> Starting 30 jobs (work_dir=/mnt/ccnas2/bdp/rz3515/projects/phism/tmp/phism/pb-flow.medium.baseline) ...
>>> Finished jacobi-1d       elapsed: 447.292012 secs   Status: 0  Error: "No Error"
>>> Finished nussinov        elapsed: 772.179239 secs   Status: 0  Error: "No Error"
>>> Finished gesummv         elapsed: 869.126733 secs   Status: 0  Error: "No Error"
>>> Finished atax            elapsed: 1202.229159 secs   Status: 0  Error: "No Error"
>>> Finished durbin          elapsed: 1291.601524 secs   Status: 0  Error: "No Error"
>>> Finished bicg            elapsed: 1625.462784 secs   Status: 0  Error: "No Error"
>>> Finished trisolv         elapsed: 1706.422457 secs   Status: 0  Error: "No Error"
>>> gemver Attempting to fix cosim issues by CosimFixStrategy(phism_directives=[], tbgen_directives=['set_directive_interface kernel_gemver A -mode ap_memory -storage_type ram_1p'], phism_mem_interfaces=[ApMemoryInterface(name='A', ports=['address0', 'ce0', 'we0', 'd0', 'q0']), ApMemoryInterf

In [7]:
pb_utils.pb_flow_runner(options["polymer"])

>>> Starting 30 jobs (work_dir=/mnt/ccnas2/bdp/rz3515/projects/phism/tmp/phism/pb-flow.small.polymer) ...
>>> Finished nussinov        elapsed: 94.725209 secs   Status: 1  Error: "Phism syn failed."
>>> Finished jacobi-1d       elapsed: 208.561463 secs   Status: 1  Error: ""
>>> bicg Attempting to fix cosim issues by CosimFixStrategy(phism_directives=[], tbgen_directives=['set_directive_interface kernel_bicg s -mode ap_memory -storage_type ram_2p'], phism_mem_interfaces=[ApMemoryInterface(name='A', ports=['address0', 'ce0', 'q0']), ApMemoryInterface(name='s', ports=['address0', 'ce0', 'we0', 'd0', 'address1', 'ce1', 'q1']), ApMemoryInterface(name='q', ports=['address0', 'ce0', 'we0', 'd0', 'q0']), ApMemoryInterface(name='p', ports=['address0', 'ce0', 'q0']), ApMemoryInterface(name='r', ports=['address0', 'ce0', 'q0'])], tbgen_mem_interfaces=[ApMemoryInterface(name='A', ports=['address0', 'ce0', 'q0']), ApMemoryInterface(name='s', ports=['address0', 'ce0', 'we0', 'd0', 'q0']), ApMemoryI

In [12]:
pb_utils.pb_flow_runner(options["polymer-baseline"])

>>> Starting 30 jobs (work_dir=/mnt/ccnas2/bdp/rz3515/projects/phism/tmp/phism/pb-flow.small.polymer-baseline) ...
>>> Finished nussinov        elapsed: 130.068559 secs   Status: 1  Error: "Cosim failed and cannot be fixed by CosimFixStrategy(phism_directives=[], tbgen_directives=[], phism_mem_interfaces=[ApMemoryInterface(name='seq', ports=['address0', 'ce0', 'q0'])], tbgen_mem_interfaces=[ApMemoryInterface(name='seq', ports=['address0', 'ce0', 'q0'])])."
>>> Finished jacobi-1d       elapsed: 219.022150 secs   Status: 1  Error: ""
>>> atax Attempting to fix cosim issues by CosimFixStrategy(phism_directives=[], tbgen_directives=['set_directive_interface kernel_atax y -mode ap_memory -storage_type ram_1p'], phism_mem_interfaces=[ApMemoryInterface(name='A', ports=['address0', 'ce0', 'q0']), ApMemoryInterface(name='x', ports=['address0', 'ce0', 'q0']), ApMemoryInterface(name='y', ports=['address0', 'ce0', 'we0', 'd0', 'q0']), ApMemoryInterface(name='tmp', ports=['address0', 'ce0', 'we0', 

## Results

Here we list all the results as a table.

In [8]:
baseline = pb_utils.to_pandas(pb_utils.process_pb_flow_result_dir(options['baseline'].work_dir, options['baseline']))
baseline

,name,phism_synth,tbgen_cosim,phism_cosim,latency,syn_latency,DSP_usage,FF_usage,LUT_usage,BRAM_18K_usage,URAM_usage,DSP_avail,FF_avail,LUT_avail,BRAM_18K_avail,URAM_avail
0,2mm,SUCCESS,COSIM_FAILED,NO_LOG,None,103510819,17,2478,2934,0,0,220,106400,53200,280,0
1,3mm,SUCCESS,COSIM_FAILED,NO_LOG,None,182400054,20,3385,3835,0,0,220,106400,53200,280,0
2,adi,SUCCESS,COSIM_FAILED,NO_LOG,None,455756701,16,3224,3690,0,0,220,106400,53200,280,0
3,atax,SUCCESS,SUCCESS,SUCCESS,331523,331523,16,1980,3202,0,0,220,106400,53200,280,0
4,bicg,SUCCESS,SUCCESS,SUCCESS,1279605,1279605,15,1971,2206,0,0,220,106400,53200,280,0
5,cholesky,SUCCESS,COSIM_FAILED,NO_LOG,None,None,26,5851,5932,0,0,220,106400,53200,280,0
6,correlation,SUCCESS,COSIM_FAILED,NO_LOG,None,None,16,3740,5475,0,0,220,106400,53200,280,0
7,covariance,SUCCESS,COSIM_FAILED,COSIM_FAILED,None,None,16,2871,4165,0,0,220,106400,53200,280,0
8,deriche,CANNOT_SYNTH,COSIM_FAILED,NO_LOG,None,None,None,None,None,None,None,None,None,None,None,None
9,doitgen,SUCCESS,COSIM_FAILED,COSIM_FAILED,None,57752001,15,1499,2223,0,0,220,106400,53200,280,0


In [7]:
polymer = pb_utils.to_pandas(pb_utils.process_pb_flow_result_dir(options['polymer'].work_dir, options['polymer']))
polymer

,name,phism_synth,tbgen_cosim,phism_cosim,latency,syn_latency,DSP_usage,FF_usage,LUT_usage,BRAM_18K_usage,URAM_usage,DSP_avail,FF_avail,LUT_avail,BRAM_18K_avail,URAM_avail
0,2mm,SUCCESS,SUCCESS,SUCCESS,625766,None,45,9546,10483,40,0,220,106400,53200,280,0
1,3mm,SUCCESS,SUCCESS,SUCCESS,1087664,None,40,11906,13454,40,0,220,106400,53200,280,0
2,adi,SUCCESS,SUCCESS,SUCCESS,1667761,None,44,8423,10213,64,0,220,106400,53200,280,0
3,atax,SUCCESS,SUCCESS,SUCCESS,32248,None,16,3252,3885,4,0,220,106400,53200,280,0
4,bicg,SUCCESS,SUCCESS,COSIM_FAILED,None,None,18,3373,3857,4,0,220,106400,53200,280,0
5,cholesky,SUCCESS,SUCCESS,SUCCESS,147170,None,29,3942,5273,256,0,220,106400,53200,280,0
6,correlation,SUCCESS,SUCCESS,COSIM_FAILED,None,None,24,11713,13513,68,0,220,106400,53200,280,0
7,covariance,SUCCESS,SUCCESS,SUCCESS,823642,None,20,7970,9528,66,0,220,106400,53200,280,0
8,deriche,SUCCESS,SUCCESS,NO_LOG,1425504,None,None,None,None,None,None,None,None,None,None,None
9,doitgen,SUCCESS,SUCCESS,SUCCESS,948551,948551,16,1683,2465,2,0,220,106400,53200,280,0


In [8]:
from collections import defaultdict

def get_latency(df, name):
  cosim_latency = df[df['name'] == name]['latency'].values[0]
  if cosim_latency:
    return cosim_latency, True
  synth_latency = df[df['name'] == name]['syn_latency'].values[0]
  return synth_latency, False

base_succ = pb_utils.filter_success(baseline)
poly_succ = pb_utils.filter_success(polymer)

names = sorted(list(set(base_succ['name']) & set(poly_succ['name'])))

data = defaultdict(list)

for name in names:
  data['name'].append(name)

  data['base_lat'].append(get_latency(base_succ, name)[0])
  data['poly_lat'].append(get_latency(poly_succ, name)[0])
  data['cosim'].append(get_latency(poly_succ, name)[1])

df = pd.DataFrame(data)
df = df.dropna()
df['poly_lat'] = [int(val) for val in df['poly_lat'].values]
df['speedup'] = (df['base_lat'] / df['poly_lat'])
df

,name,base_lat,poly_lat,cosim,speedup
0,2mm,2147219,625766,True,3.431345
1,3mm,4320054,1087664,True,3.971864
2,adi,15715801,1667761,True,9.423293
3,atax,32259,32248,True,1.000341
4,cholesky,5408942,147170,True,36.753020
5,covariance,598006,823642,True,0.726051
6,doitgen,3623001,948551,True,3.819511
7,durbin,30942,19080,True,1.621698
8,fdtd-2d,4524681,11123253,True,0.406777
9,floyd-warshall,11664024,13284361,True,0.878027


In [9]:
df['speedup'].mean()

5.024632413599304